# Upload Clara Models to Hugging Face Hub

This notebook uploads all Clara models from Google Drive to Hugging Face Hub.

**Models:**
- `clara-knowledge` - Phi-3 fine-tuned knowledge brain (~7GB)
- `clara-warmth` - Mistral LoRA adapter (~52MB)
- `clara-playful` - Mistral LoRA adapter (~52MB)
- `clara-encouragement` - Mistral LoRA adapter (~52MB)

In [ ]:
# Cell 1: Install dependencies
!pip install -q huggingface_hub

In [ ]:
# Cell 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Cell 3: Login to Hugging Face
# Get your token from: https://huggingface.co/settings/tokens
# Make sure it has WRITE access!

from huggingface_hub import login
login()

In [ ]:
# Cell 4: Configuration
# UPDATE THESE PATHS to match your Google Drive structure!

import os

# Your Hugging Face username
HF_USERNAME = "ChrisHartline"  # Change if different

# Base path in Google Drive
DRIVE_BASE = "/content/drive/MyDrive/Lily/models"

# Model configurations: (drive_folder, hf_repo_name, description)
MODELS = [
    {
        "drive_path": f"{DRIVE_BASE}/clara-knowledge",
        "hf_repo": f"{HF_USERNAME}/clara-knowledge",
        "description": "Clara Knowledge Brain - Phi-3 fine-tuned for medical, coding, teaching, quantum domains",
        "tags": ["phi-3", "clara", "fine-tuned", "knowledge"]
    },
    {
        "drive_path": f"{DRIVE_BASE}/mistral_warmth",
        "hf_repo": f"{HF_USERNAME}/clara-warmth",
        "description": "Clara Warmth Adapter - LoRA for warm, friendly personality",
        "tags": ["mistral", "lora", "clara", "personality", "warmth"]
    },
    {
        "drive_path": f"{DRIVE_BASE}/mistral_playful",
        "hf_repo": f"{HF_USERNAME}/clara-playful",
        "description": "Clara Playful Adapter - LoRA for playful, witty personality",
        "tags": ["mistral", "lora", "clara", "personality", "playful"]
    },
    {
        "drive_path": f"{DRIVE_BASE}/mistral_encouragement",
        "hf_repo": f"{HF_USERNAME}/clara-encouragement",
        "description": "Clara Encouragement Adapter - LoRA for supportive, motivating personality",
        "tags": ["mistral", "lora", "clara", "personality", "encouragement"]
    },
]

# Verify paths exist
print("Checking paths...")
for model in MODELS:
    path = model["drive_path"]
    exists = os.path.exists(path)
    status = "✅" if exists else "❌"
    print(f"  {status} {path}")
    if exists:
        files = os.listdir(path)
        print(f"      Files: {len(files)}")

In [ ]:
# Cell 5: Upload function
from huggingface_hub import HfApi, create_repo
import shutil

api = HfApi()

def upload_model(drive_path, hf_repo, description, tags):
    """
    Upload a model folder to Hugging Face Hub.
    Skips checkpoint folders (training artifacts).
    """
    print(f"\n{'='*60}")
    print(f"Uploading: {hf_repo}")
    print(f"From: {drive_path}")
    print(f"{'='*60}")
    
    if not os.path.exists(drive_path):
        print(f"❌ Path not found: {drive_path}")
        return False
    
    # Create repo if it doesn't exist
    try:
        create_repo(hf_repo, repo_type="model", exist_ok=True)
        print(f"✅ Repo created/exists: {hf_repo}")
    except Exception as e:
        print(f"⚠️ Repo creation: {e}")
    
    # Create a clean copy without checkpoints
    temp_path = f"/content/temp_upload"
    if os.path.exists(temp_path):
        shutil.rmtree(temp_path)
    os.makedirs(temp_path)
    
    # Copy files (skip checkpoint folders)
    for item in os.listdir(drive_path):
        src = os.path.join(drive_path, item)
        dst = os.path.join(temp_path, item)
        
        # Skip checkpoint folders
        if item.startswith("checkpoint-"):
            print(f"  ⏭️ Skipping: {item}")
            continue
        
        if os.path.isfile(src):
            shutil.copy2(src, dst)
            size_mb = os.path.getsize(src) / (1024*1024)
            print(f"  📄 {item} ({size_mb:.1f} MB)")
        elif os.path.isdir(src):
            shutil.copytree(src, dst)
            print(f"  📁 {item}/")
    
    # Create model card
    model_card = f"""---
tags:
{chr(10).join(f'- {tag}' for tag in tags)}
license: mit
---

# {hf_repo.split('/')[-1]}

{description}

## Usage

```python
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("{hf_repo}")
tokenizer = AutoTokenizer.from_pretrained("{hf_repo}")
```

## Part of Clara

This model is part of the Clara embodied AI system.
See: https://github.com/ChrisHartline/embodiedMoE
"""
    
    with open(os.path.join(temp_path, "README.md"), "w") as f:
        f.write(model_card)
    
    # Upload
    print(f"\n📤 Uploading to {hf_repo}...")
    try:
        api.upload_folder(
            folder_path=temp_path,
            repo_id=hf_repo,
            repo_type="model",
            commit_message=f"Upload {hf_repo.split('/')[-1]}"
        )
        print(f"✅ Success! https://huggingface.co/{hf_repo}")
        return True
    except Exception as e:
        print(f"❌ Upload failed: {e}")
        return False
    finally:
        # Cleanup
        if os.path.exists(temp_path):
            shutil.rmtree(temp_path)

In [ ]:
# Cell 6: Upload LoRA adapters first (smaller, faster)
# These are the personality adapters (~52MB each)

lora_models = [m for m in MODELS if "lora" in m.get("tags", [])]

print(f"Uploading {len(lora_models)} LoRA adapters...\n")

for model in lora_models:
    upload_model(
        model["drive_path"],
        model["hf_repo"],
        model["description"],
        model["tags"]
    )

In [ ]:
# Cell 7: Upload Knowledge Brain (large ~7GB, may take a while)
# Run this cell separately - it takes longer

knowledge_model = [m for m in MODELS if "knowledge" in m.get("tags", [])][0]

print("Uploading Knowledge Brain (~7GB - this may take 10-20 minutes)...")
print("☕ Go grab a coffee!\n")

upload_model(
    knowledge_model["drive_path"],
    knowledge_model["hf_repo"],
    knowledge_model["description"],
    knowledge_model["tags"]
)

In [ ]:
# Cell 8: Verify uploads
print("\n" + "="*60)
print("UPLOAD SUMMARY")
print("="*60)

for model in MODELS:
    repo = model["hf_repo"]
    try:
        info = api.model_info(repo)
        print(f"\n✅ {repo}")
        print(f"   URL: https://huggingface.co/{repo}")
        print(f"   Files: {len(info.siblings)}")
    except Exception as e:
        print(f"\n❌ {repo} - Not found or error: {e}")

print("\n" + "="*60)
print("Done! Update your backend to load from HuggingFace.")
print("="*60)